# Publishing a Dataset to the Web Project

### By Cillian Murphy


## Project Brief

This project focuses on publishing data to the web (LO5).  It will use the data you have captured in project 2 and display it on a web site.Based on the project brief for project 2 shown below, you should have the following information:

The core information you should collect is:
- Job Title
- Company
- Location (City at a minimum)
- Date when job was listed

Data you should try and obtain where it is included in a job specification:
- Salary (or salary range)
- Skills required
- Years experience required


Based on this you will produce a web site that displays analysis of the data you have captured. Exactly what data and analysis you present depends on the data you have captured. As a guideline, include the following:

- Multiple pages, that can be easily navigated.
- Charts and/or tables that show aggregate data of current job vacancies.
- Charts and/or tables that show historical information about jobs vacancies.
- Shows detailed information about current and historical jobs.
- Your source data should be stored in a relational database (e.g. MySql).
- You may combine database queries with Pandas to perform analysis when preparing data on the server.

## 1. Design

https://www.figma.com/design/WKLdJvYYVwWcgRpAb8LKB7/Wireframe_design_CM_CA3?m=auto&t=I6Al2tknzW47bQWs-6

## 2. DataCleaning for current jobs

In [1]:
# imports
import pandas as pd
import numpy as np
import mysql.connector
import csv


In [2]:
class DataCleaner:
    def __init__(self, next: 'DataCleaner' = None):  
        ''' initialise the next handler reference'''
        self.next = next
        
    def process(self, data):
        ''' call process_task on the concrete class
        then calls the next handler'''
        self.process_task(data)
        if self.next is not None:
            self.next.process(data)
    
    def process_task(self, data):
        ''' abstract method'''
        pass
    
    def add_datacleaner(self, datacleaner):
        '''Adds a new handler to the end of the chain '''
        if self.next != None:
            self.next.add_datacleaner(datacleaner)
        else:
            self.next = datacleaner

In [ ]:
class CleanData(DataCleaner):
    def process_task(self, data):

        data["salary"] = data["salary"].str.replace("s\n"," ")

        # Salary and experience were joined by the circle so had to replace this with a comma
        data["salary"] = data["salary"].str.replace("•",", ")

        # Creating the experience column by taking where it had the experience in the salary column and assigning that to experience
        data["experience"] = data["salary"].str.extract(r'((?:\d+\s*-\s*\d+\s*Years)|(?:Fresher))')

        # Finding where it had the salary per month part and assigning that to salary column
        data['salary'] = data['salary'].str.extract(r'((?:USD|SGD)\s[\d,]+\s*-\s*[\d,]+\s*per\s*month)')

        # Assigning each experience level to a key value then looping through the experience column to assign the value to each type of experience category
        exp_data = {"Fresher":1,"3 - 5 Years":4,"5 - 7 Years":6,"2 - 4 Years":3,"1 - 3 Years":2,"2 - 5 Years":3,"5 - 8 Years":6,"4 - 6 Years":5,"3 - 8 Years":6,"10 - 12 Years":11,"8 - 10 Years":9,"1 - 5 Years":3,"5 - 10 Years":7,"0 - 2 Years":1}
        for exp in exp_data:
            data.loc[data["experience"].str.contains(exp,na = False), "experience"] = exp_data[exp]

        # Extracting what currency the job is listed in and assigning it to a new column currency
        data["currency"] = data["salary"].str.extract(r'((?:USD|SGD))')

        # Getting rid of any white space in the salary column
        data["salary"] = data["salary"].str.strip()

        # Gettiong rid of the hyphon between the salaries
        data["salary"] = data["salary"].str.replace("-"," ")

        # Getting rid of the USD/SGD currency in the salary column as we have a new column with that info
        data['salary'] = data['salary'].str.replace(r'(SGD|USD|\s*per month)', '', regex=True)

        # Splitting the salary column to max and min salaries to compute the median
        data[["min_salary","max_salary"]] = data["salary"].str.split(expand=True)

        # Getting rid of commas so that I can convert the salaries to floats
        data['min_salary'] = data['min_salary'].str.replace(',', '').astype(float)
        data['max_salary'] = data['max_salary'].str.replace(',', '').astype(float)

        # Computing the median salary then writing that to a new column called median salary
        data['median_salary'] = data[['min_salary', 'max_salary']].median(axis=1)

        # Drop salary column as have all info neccesary within min, max and median salary
        data.drop(columns=["salary"],inplace = True)

        # Finding rows where job title was null as the webscraper didnt obtain that data and dropping these rows
        indexdata = data[(data['job_title'].isna() == True)].index
        data.drop(indexdata,inplace=True)
        
        return data.to_csv("new_jobs.csv")

In [ ]:
class CleanSkills(DataCleaner):
    def process_task(self, data):
        skills = pd.DataFrame(data["skills"])

        # Getting rid of the /n in between each skill
        skills["skills"] = skills["skills"].str.replace("\n",", ")

        # Splitting the skills every time there is a comma while still being assigned to the job_id
        skills = skills.assign(skills=skills['skills'].str.split(',')).explode('skills')
        skills['skills'] = skills['skills'].str.strip()

        # Dropping null rows
        indexdata = skills[(skills['skills'].isna() == True)].index
        skills.drop(indexdata,inplace=True)

        # dropping skills from original data
        data.drop(columns=["skills"],inplace=True)

        # Writing the skills data to a csv
        return skills.to_csv("new_skills.csv")
        


In [ ]:
# Variable names for data that will be passed through the data cleaning pipeline
data = pd.read_csv("current_jobs.csv")
data2 = pd.read_csv("table_data.csv")

In [ ]:
# Setting up chain calling cleanskills first then passing that data through to the next handler
chain = CleanSkills(CleanData())
chain.process(data)

## 3. Writing to Database

In [ ]:
# Creating variables for the cleaned data to pass to my database 
new_skills_data = pd.read_csv("new_skills.csv")
new_jobs_data = pd.read_csv("new_jobs.csv")

old_skills_data = pd.read_csv("skills_2_2_25.csv")
old_jobs_data = pd.read_csv("jobs_2_2_25.csv")

In [ ]:
def create_database():
    # Creating connection to the database
    mydb = mysql.connector.connect(user='#', password='#', host='#')

    # Creating cursor to interact with my database
    cur = mydb.cursor()

    cur.execute("CREATE DATABASE IF NOT EXISTS Jobs_Data")


    # Connecting to my database
    cur.execute("USE Jobs_Data")


    cur.execute("""
    CREATE TABLE IF NOT EXISTS new_jobs (
        job_id INT PRIMARY KEY,
        job_title VARCHAR(255),
        location VARCHAR(255),
        min_salary FLOAT,
        median_salary FLOAT,
        max_salary FLOAT,
        company_name VARCHAR(255),
        date_posted VARCHAR(255),
        experience FLOAT
    )
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS new_skills (
        job_id INT,
        skill_name VARCHAR(255),
        FOREIGN KEY (job_id) REFERENCES new_jobs(job_id)
    )
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS old_jobs (
        job_id INT PRIMARY KEY,
        job_title VARCHAR(255),
        location VARCHAR(255),
        min_salary FLOAT,
        median_salary FLOAT,
        max_salary FLOAT,
        company_name VARCHAR(255),
        date_posted VARCHAR(255),
        experience FLOAT
    )
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS old_skills (
        job_id INT,
        skill_name VARCHAR(255),
        FOREIGN KEY (job_id) REFERENCES new_jobs(job_id)
    )
    """)



create_database()

In [ ]:
def insert_new_data(job_data,skills_data):
    mydb = mysql.connector.connect(user='#', password='#', host='#',database='Jobs_Data')
    cur = mydb.cursor()

    # renaming columns to match the sql table
    skills_data = skills_data.rename(columns={"Unnamed: 0":"job_id","skills":"skill_name"})

    job_data = job_data.rename(columns={"Unnamed: 0":"job_id"})

    # looping through job data and adding the data to my database
    for index, row in job_data.iterrows():
        cur.execute("INSERT INTO new_jobs (job_id,job_title,location,min_salary,median_salary,max_salary,company_name,date_posted,experience) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)",(row["job_id"],row["job_title"],row["location"],row["min_salary"],row["median_salary"],row["max_salary"],row["company_name"],row["date_posted"],row["experience"]))
        mydb.commit()

    # Filter skills_data to include only rows with job_id present in job_data
    valid_job_ids = set(job_data["job_id"])
    filtered_skills_data = skills_data[skills_data["job_id"].isin(valid_job_ids)]

    for index, row in filtered_skills_data.iterrows():
        cur.execute("INSERT INTO new_skills (job_id,skill_name) VALUES (%s,%s)",(row["job_id"],row["skill_name"]))
        mydb.commit()

    mydb.close()    

# passing the csv file into my database
insert_new_data(new_jobs_data,new_skills_data)    


    

In [23]:
def insert_old_data(job_data,skills_data):
    mydb = mysql.connector.connect(user='#', password='#', host='#',database='Jobs_Data')
    cur = mydb.cursor()

    skills_data = skills_data.rename(columns={"Unnamed: 0":"job_id","skills":"skill_name"})

    job_data = job_data.rename(columns={"Unnamed: 0":"job_id"})

    for index, row in job_data.iterrows():
        cur.execute("INSERT INTO old_jobs (job_id,job_title,location,min_salary,median_salary,max_salary,company_name,date_posted,experience) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)",(row["job_id"],row["job_title"],row["location"],row["min_salary"],row["median_salary"],row["max_salary"],row["company_name"],row["date_posted"],row["experience"]))
        mydb.commit()

    # Filter skills_data to include only rows with job_id present in job_data
    valid_job_ids = set(job_data["job_id"])
    filtered_skills_data = skills_data[skills_data["job_id"].isin(valid_job_ids)]

    for index, row in filtered_skills_data.iterrows():
        cur.execute("INSERT INTO old_skills (job_id,skill_name) VALUES (%s,%s)",(row["job_id"],row["skill_name"]))
        mydb.commit()

    mydb.close()    

insert_old_data(old_jobs_data,old_skills_data)   


## 4. Generating Data for graphs

### Current Job Vacancy Data 
- Graph 1 Salary
- Graph 2 Jobs Posted per Date
- Graph 3 Jobs listed per company

In [ ]:
# Function to query the data and write to a new csv, by passing in the query to run and the filename to write it to
def get_data(query,filename):
    try:
        # Coneecting to database and setting up my cursor for databse interaction
        mydb = mysql.connector.connect(user='#', password='#', host='#',database='Jobs_Data')

        cur = mydb.cursor()

        # executing the query that has been passed in
        cur.execute(query)    

        # creating a empty list for writing the results to
        data = []

        # looping through the results, adding that to the list and finally writing to the csv file
        results = cur.fetchall()
        for res in results:
            data.append(res)
        with open(filename,"w",newline="") as f:
            writer = csv.writer(f)
            writer.writerows(data)

    # Catching any errors in database connection or problems with the query/file
    except mysql.connector.Error as er:
        print("Something went wrong {}".format(er))

    finally:
        if mydb is not None and mydb.is_connected():
            mydb.close()
            print('Database connection closed.')    

# Graph 1 Salarys
get_data("SELECT min_salary,median_salary,max_salary from new_jobs WHERE min_salary is NOT null and median_salary is not null and max_salary is not null ORDER by max_salary desc","salary_data.csv")

# Graph 2 Jobs posted per date
get_data("SELECT COUNT(job_title) as jobs_per_date,date_posted from new_jobs GROUP by date_posted order by COUNT(job_title) desc","jobs_per_date.csv")

# Graph 3 Jobs listed per company
get_data("SELECT COUNT(job_title) as jobs_per_company,company_name from new_jobs where company_name is not null GROUP by company_name order by COUNT(job_title) DESC LIMIT 10","jobs_per_company.csv")

# Past Salary Data
get_data("SELECT min_salary,median_salary,max_salary from old_jobs WHERE min_salary is NOT null and median_salary is not null and max_salary is not null ORDER by max_salary desc","old_jobs_salaries.csv")

# Top 10 current skills
get_data("SELECT COUNT(skill_name),skill_name from new_skills GROUP by skill_name ORDER by COUNT(skill_name) desc LIMIT 10","top_10_current_skills.csv")

# Top 10 old skills
get_data("SELECT COUNT(skill_name),skill_name from old_skills GROUP by skill_name ORDER by COUNT(skill_name) desc LIMIT 10","top_10_old_skills.csv")

# Past experience needed
get_data("SELECT experience,count(experience) as experience_needed from new_jobs where experience is NOT null GROUP by experience ORDER by COUNT(experience) DESC","new_experience_needed.csv")

# Current Experience needed
get_data("SELECT experience,count(experience) as experience_needed from old_jobs where experience is NOT null GROUP by experience ORDER by COUNT(experience) DESC","old_experience_needed.csv")

# Highest Paying Skills
get_data("SELECT skill_name, median_salary from new_skills join new_jobs on new_skills.job_id = new_jobs.job_id WHERE median_salary is NOT null GROUP by skill_name ORDER by median_salary DESC LIMIT 25","top_20_paying_skills.csv")

# Companys looking for the most skills
get_data("SELECT company_name,COUNT(skill_name) as amount_of_skills from new_jobs JOIN new_skills on new_skills.job_id = new_jobs.job_id WHERE company_name is not null GROUP by company_name ORDER by COUNT(skill_name) desc LIMIT 25;","company_skills.csv")

# Different levels of experience
get_data("SELECT experience,COUNT(experience) as experience_needed from new_jobs GROUP by experience ORDER by experience desc","experience_data.csv")

# Average experience level needed per company
get_data("SELECT company_name,avg(experience) as average_experience from new_jobs WHERE experience is not null GROUP by company_name ORDER by AVG(experience) desc","avg_exp.csv")



Database connection closed.
